In [1]:
## caged_brach
%reload_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from paths import *
from scpts import manipulation
from scpts import io
from scpts import caged_novo

from basedosdados import Storage, Table, Dataset
import basedosdados as bd

import glob


### Download Dados
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED/
 - ftp://ftp.mtps.gov.br/pdet/microdados/CAGED_AJUSTES/
 - ftp://ftp.mtps.gov.br/pdet/microdados/NOVO%20CAGED/

### Get Urls

In [88]:
download_dict = caged_novo.get_download_links()

estabelecimentos
movimentacoes


In [15]:
download_dict

{'estabelecimentos': {'must_download': ['Estabelecimentos/2021/Julho/CAGEDESTAB202007.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202008.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202009.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202010.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202011.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202012.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202101.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202102.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202103.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202104.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202105.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202106.7z',
   'Estabelecimentos/2021/Julho/CAGEDESTAB202107.7z'],
  'check_download': {'202001': 'Estabelecimentos/2021/Janeiro/CAGEDESTAB202001.7z',
   '202002': 'Estabelecimentos/2021/Fevereiro/CAGEDESTAB202002.7z',
   '202003': 'Estabelecimentos/2021/Março/CAGEDESTAB202003.7z',
   '202004': 'Estabelecimentos/2021/Abril/CA

### Download Data

- download do zip na pasta raw

In [143]:
RAW_PATH = "../data/caged_novo/raw/"
creat_path_tree(RAW_PATH)
for tipo in download_dict.keys():
    download_opt = download_dict[tipo]["check_download"]
    download_opt.update(download_dict[tipo]["must_download"])
    # TODO: etapa de filtragem do que n deve ser baixado

    for year_month_path in list(download_opt.keys()):
        print(tipo, year_month_path)
        save_path = RAW_PATH + f"{tipo}/" + year_month_path
        download_data(save_path, download_opt[year_month_path])
    print("\n")

estabelecimentos 2020/1/
estabelecimentos 2020/2/
estabelecimentos 2020/3/
estabelecimentos 2020/4/
estabelecimentos 2020/5/
estabelecimentos 2020/6/


movimentacoes 2020/1/
movimentacoes 2020/2/
movimentacoes 2020/3/
movimentacoes 2020/4/
movimentacoes 2020/5/
movimentacoes 2020/6/




### TRATAMENTO

- extrair
- load
- padroniza nome das colunas
- limpa
- salva, formato ano=xxxx/mes=xx/sigla_uf=XX

In [337]:
import py7zr
from scpts import manipulation


def extract_file(file_path, file_name, save_rows=10):
    print(file_path)
    if not os.path.exists(f"{file_path}{file_name}.csv"):
        archive = py7zr.SevenZipFile(f"{file_path}{file_name}.7z", mode="r").extractall(
            path=file_path
        )
        filename_txt = [
            file for file in os.listdir(file_path) if ".txt" in file.lower()
        ][0]

        #         print(filename_txt)

        df = pd.read_csv(
            f"{file_path}{filename_txt}",
            sep=";",
            encoding="latin-1",
            nrows=save_rows,
            dtype="str",
        )

        #         df.columns = manipulation.normalize_cols(df.columns)

        df.to_csv(f"{file_path}{file_name}.csv", index=False, encoding="latin-1")

        os.remove(f"{file_path}{filename_txt}")


def clean_csvs(file_path, file_name):
    try:
        os.remove(f"{file_path}{file_name}.csv")
    except:
        pass


def rename_add_orginaze_columns(file_path, file_name, tipo):
    municipios = pd.read_csv("../data/caged_novo/diretorio_municipios.csv")
    df = pd.read_csv(f"{file_path}{file_name}.csv")

    colunas_estabelecimento = {
        "sigla_uf": "sigla_uf",
        "id_municipio": "id_municipio",
        "município": "id_municipio_6",
        "seção": "cnae_2",
        "subclasse": "cnae_2_subclasse",
        "admitidos": "admitidos",
        "desligados": "desligados",
        "fonte_desl": "fonte_desligamento",
        "saldomovimentação": "saldo_movimentacao",
        "tipoempregador": "tipo_empregador",
        "tipoestabelecimento": "tipo_estabelecimento",
        "tamestabjan": "tamanho_estabelecimento_janeiro",
        "competência": "",
        "região": "",
        "uf": "",
    }

    colunas_movimentacoes = {
        "sigla_uf": "sigla_uf",
        "id_municipo": "id_municipio",
        "município": "id_municipio_6",
        "seção": "cnae_2",
        "subclasse": "cnae_2_subclasse",
        "cbo2002ocupação": "cbo_2002",
        "saldomovimentação": "saldo_movimentacao",
        "categoria": "categoria",
        "graudeinstrução": "grau_instrucao",
        "idade": "idade",
        "horascontratuais": "horas_contratuais",
        "raçacor": "raca_cor",
        "sexo": "sexo",
        "tipoempregador": "tipo_empregador",
        "tipoestabelecimento": "tipo_estabelecimento",
        "tipomovimentação": "tipo_movimentacao",
        "tipodedeficiência": "tipo_deficiencia",
        "indtrabintermitente": "indicador_trabalho_intermitente",
        "indtrabparcial": "indicador_trabalho_parcial",
        "salário": "salario_mensal",
        "tamestabjan": "tamanho_estabelecimento_janeiro",
        "indicadoraprendiz": "indicador_aprendiz",
        "fonte": "fonte",
        "competência": "",
        "região": "",
        "uf": "",
    }

    if tipo == "estabelecimentos":
        col_dict = colunas_estabelecimento
    else:
        col_dict = colunas_movimentacoes

    remove_cols = [col for col in col_dict if col_dict.get(col) == ""]

    df = df.drop(columns=remove_cols)
    df = df.rename(columns=col_dict)
    df = df.merge(municipios, on="id_municipio_6", how="left")

    col_order = [col_dict[col] for col in col_dict if col not in remove_cols]
    df = df[col_order]

    return df

In [338]:
RAW_PATH = "../data/caged_novo/raw/"
CLEAN_PATH = "../data/caged_novo/clean/"

dd = pd.DataFrame()
for tipo in list(download_dict.keys())[1:]:
    print(tipo)
    download_opt = download_dict[tipo]["check_download"]
    download_opt.update(download_dict[tipo]["must_download"])
    # TODO: etapa de filtragem do que n deve ser baixado
    for year_month_path in list(download_opt.keys()):
        file_name = download_opt[year_month_path].split("/")[-1].split(".")[0]
        file_path = RAW_PATH + f"{tipo}/" + year_month_path

        # extrai arquivo
        extract_file(file_path, file_name, save_rows=10)

        # load e organiza os dados
        df = rename_add_orginaze_columns(file_path, file_name, tipo)

        # limpa dados

        # salva no formato de particao

        #         clean_csvs(file_path,file_name)


movimentacoes
../data/caged_novo/raw/movimentacoes/2020/1/
../data/caged_novo/raw/movimentacoes/2020/2/
../data/caged_novo/raw/movimentacoes/2020/3/
../data/caged_novo/raw/movimentacoes/2020/4/
../data/caged_novo/raw/movimentacoes/2020/5/
../data/caged_novo/raw/movimentacoes/2020/6/
../data/caged_novo/raw/movimentacoes/2020/7/
../data/caged_novo/raw/movimentacoes/2020/8/
../data/caged_novo/raw/movimentacoes/2020/9/
../data/caged_novo/raw/movimentacoes/2020/10/
../data/caged_novo/raw/movimentacoes/2020/11/
../data/caged_novo/raw/movimentacoes/2020/12/
../data/caged_novo/raw/movimentacoes/2021/1/
../data/caged_novo/raw/movimentacoes/2021/2/
../data/caged_novo/raw/movimentacoes/2021/3/
../data/caged_novo/raw/movimentacoes/2021/4/
../data/caged_novo/raw/movimentacoes/2021/5/
../data/caged_novo/raw/movimentacoes/2021/6/
../data/caged_novo/raw/movimentacoes/2021/7/
